Assignment 6

In [97]:
import numpy as np

stuff=np.load("data.npz")
X_trn = stuff["X_trn"]
y_trn = stuff["y_trn"]
X_tst = stuff["X_tst"]
# no Y_tst !

In [98]:
# CSV Helper Function

# import numpy as np
# import csv

# def write_csv(y_pred, filename):
#     """Write a 1d numpy array to a Kaggle-compatible .csv file"""
#     with open(filename, 'w') as csv_file:
#         csv_writer = csv.writer(csv_file)
#         csv_writer.writerow(['Id', 'Category'])
#         for idx, y in enumerate(y_pred):
#             csv_writer.writerow([idx, y])


# data = np.load('data.npz')
# X_tst = data['X_tst']
# y_pred = np.random.randint(0, 3, size=len(X_tst)) # random predictions
# write_csv(y_pred, 'sample_predictions.csv')

In [99]:
# # Example plot

# from matplotlib import pyplot as plt

# def show(x):
#     img = x.reshape((3,29,29)).transpose(1,2,0)
#     plt.imshow(img)
#     plt.axis('off')
#     plt.draw()
#     plt.pause(0.01)

# show(X_trn[7])

Question 1

In [100]:
X = [1.0, 2.0, 3.0, 4.0, 5.0]
Y = [1, 1, 0, 0, 1]
splits = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]

# calculate parent entropy

parent_p1 = sum(Y) / len(Y)
parent_p0 = 1 - parent_p1
parent_entropy = (-parent_p1*np.log2(parent_p1)) + (- parent_p0*np.log2(parent_p0))

# calculate child entropy for each split
child_entropy = {.5: -1, 1.5: -1, 2.5: -1, 3.5: -1, 4.5: -1, 5.5: -1}

for split in splits:
    # split data
    left = []
    right = []
    for i in range(len(X)):
        if X[i] < split:
            left.append(Y[i])
        else:
            right.append(Y[i])

    # calculate I(p1) and I(p2) for both left and right sides (if empty, set to 0)
    left_p1 = 0
    if len(left) != 0:
        left_p1 = sum(left) / len(left)
    left_p0 = 1 - left_p1

    right_p1 = 0
    if len(right) != 0:
        right_p1 = sum(right) / len(right)
    right_p0 = 1 - right_p1

    if left_p1 == 0:
        left_entropy = -left_p0*np.log2(left_p0)
    elif left_p0 == 0:
        left_entropy = -left_p1*np.log2(left_p1)
    else:
        left_entropy = (-left_p1*np.log2(left_p1)) + (- left_p0*np.log2(left_p0))
    
    if right_p1 == 0:
        right_entropy = -right_p0*np.log2(right_p0)
    elif right_p0 == 0:
        right_entropy = -right_p1*np.log2(right_p1)
    else:
        right_entropy = (-right_p1*np.log2(right_p1)) + (- right_p0*np.log2(right_p0))
    
    # calculate child entropy and store
    child_entropy[split] = ((len(left))*left_entropy + (len(right))*right_entropy) / len(Y)

# calculate information gain and print
information_gain = {split: parent_entropy - child_entropy[split] for split in splits}

print("Split | Information Gain")
for split in splits:
    print(f" {split}  |  {information_gain[split]}")

Split | Information Gain
 0.5  |  0.0
 1.5  |  0.17095059445466854
 2.5  |  0.4199730940219748
 3.5  |  0.01997309402197489
 4.5  |  0.17095059445466854
 5.5  |  0.0


Question 2

In [101]:

class ClassificationStump():
    def __init__(self):
        return

    def fit(self, X_trn, y_trn):
        
        # do stuff here
        D = len(X_trn[0])
        N = len(X_trn)
        dim = -1
        thresh = -1
        min_error = np.inf
        c_left = 0
        c_right = 0

        for i in range(D):

            sorted_indices = np.argsort(X_trn[:, i])
            Z = X_trn[sorted_indices]
            y_sorted = y_trn[sorted_indices]


            for n in range(N-1):
                t = (Z[n][i] + Z[n+1][i])/2

                R1 = y_sorted[:n+1] # x <= t
                R2 = y_sorted[n+1:] # x > t

                R1_count = np.bincount(R1, minlength=np.max(y_sorted) + 1)
                R2_count = np.bincount(R2, minlength=np.max(y_sorted) + 1)
                
                c1 = np.argmax(R1_count)
                c2 = np.argmax(R2_count)

                p1 = R1_count / len(R1) # probability of each class in R1
                p2 = R2_count / len(R2) # probability of each class in R2

                gini_left = np.sum(p1 * (1-p1)) # sum pi(1-pi) for each class in R1
                gini_right = np.sum(p2 * (1-p2))
                
                gini_total = (len(R1) / N) * gini_left + (len(R2) / N) * gini_right # weighted average of gini impurity
                
                error = gini_total
                if error < min_error:
                    min_error = error
                    dim = i
                    c_left = c1
                    c_right = c2
                    thresh = t

        self.model = (dim, thresh, c_left, c_right)
        self.model = {'dim': dim, 'thresh': thresh, 'c_left': c_left, 'c_right': c_right}
        return 

    def predict(self, X_val, y_val=None):
        assert hasattr(self, "model"), "No fitted model!"
        # do stuff here (use self.model for prediction)
        y_pred = []
        for x_val in X_val:
            y_pred.append(self.model['c_left'] if x_val[self.model['dim']] <= self.model['thresh'] else self.model['c_right'])
        
        return y_pred



In [102]:

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
X_tst = data['X_tst']

clf = ClassificationStump()


X_trn = X_trn.reshape((6000, 3 * 29 * 29))


clf.fit(X_trn, y_trn)
y_pred = clf.predict(X_trn)
correct_pred = sum(y_pred == y_trn)
e = 1 - correct_pred / len(y_pred)
print("Training classification error: ", e)



Training classification error:  0.6421666666666667


Question 3

In [103]:
# train classifcation trees
from sklearn.tree import DecisionTreeClassifier

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']

X_trn = X_trn.reshape((len(X_trn), -1))

max_depths = [1,3,6,9,12,14]
# return a 6x1 table of classification errors
print(" DEPTH | TRAINING ERROR")
for max_depth in max_depths:
    clf = DecisionTreeClassifier(max_depth=max_depth)
    clf.fit(X_trn, y_trn)
    y_pred = clf.predict(X_trn)
    correct_pred = sum(y_pred == y_trn)
    acc = correct_pred / len(y_trn)
    e = 1 - acc 
    print(f"  {max_depth}  | {e}")

 DEPTH | TRAINING ERROR
  1  | 0.6433333333333333
  3  | 0.5515
  6  | 0.4033333333333333
  9  | 0.2493333333333333
  12  | 0.1293333333333333
  14  | 0.07150000000000001


Question 4

Question 5

Question 6

In [104]:
# linear classifier model with logistic loss and ridge regularization only using sklearn.linear and decision_function() method

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
X_tst = data['X_tst']

X_trn = X_trn.reshape((6000, 3*29*29))  


lambda_vals = [0.1, 1, 10, 100, 1000]

# report training classification error and logistic loss for each lambda value
print(" LAMBDA | TRAINING ERROR | LOGISTIC LOSS")
for lambda_val in lambda_vals:
    clf = LogisticRegression(penalty='l2', C=1/lambda_val, max_iter=10000, tol=.001)
    clf.fit(X_trn, y_trn)
    
    confidence_scores = clf.decision_function(X_trn) # get confidence score per class per sample 
   
    exp_scores = np.exp(confidence_scores) # prepare for softmax
    sum_exp_scores_per_sample = [sum(scores) for scores in exp_scores]

    probs = [exp_scores[i] / sum_exp_scores_per_sample[i] for i in range(len(exp_scores))] # softmax to get probs
    y_pred = [np.argmax(prob) for prob in probs] # get prediction from highest prob
    logistic_loss = log_loss(y_trn, probs) # calculate logistic loss
   
    correct_preds = np.sum(y_pred == y_trn)
    e = 1 - (correct_preds / len(y_trn)) # calculate classification error
    print(f"  {lambda_val}  | {e} | {logistic_loss}")


 LAMBDA | TRAINING ERROR | LOGISTIC LOSS
  0.1  | 0.17000000000000004 | 0.4771340095214954
  1  | 0.21383333333333332 | 0.582162089440951
  10  | 0.26449999999999996 | 0.6939481457195188
  100  | 0.3031666666666667 | 0.7863802988369172
  1000  | 0.33666666666666667 | 0.877910964337898


Question 7

In [105]:
# K nearest neighbors classifier
from sklearn.neighbors import KNeighborsClassifier

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
X_tst = data['X_tst']

X_trn = X_trn.reshape((len(X_trn), -1))

k_neighbors = [1, 3, 5, 7, 9, 11]

print(" K | TRAINING ERROR")
for k in k_neighbors:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_trn, y_trn)
    y_pred = clf.predict(X_trn)
    correct_pred = sum(y_pred == y_trn)
    acc = correct_pred / len(y_trn)
    e = 1 - acc
    print(f" {k} | {e}")

 K | TRAINING ERROR
 1 | 0.0
 3 | 0.29200000000000004
 5 | 0.31633333333333336
 7 | 0.3478333333333333
 9 | 0.3626666666666667
 11 | 0.3663333333333333


Question 8

PART 2 - Cross Validation

Question 9

In [106]:
from sklearn.model_selection import KFold

class Classifier():
    def __init__(self, model):
        self.model = model
        return

    def fit(self, X_trn, y_trn):
        self.model.fit(X_trn, y_trn)
        # self.model is stored
        return

    def predict(self, X_val, y_val=None):
        # self.model is used
        y_pred = self.model.predict(X_val)
        return y_pred


def cross_validation(classifier, X_trn, y_trn, n_folds=5):
    # do stuff here

    kf = KFold(n_splits=n_folds)
    outputs = []

    for train_index, test_index in kf.split(X_trn):
        
        x_train, x_test = X_trn[train_index], X_trn[test_index]
        y_train, y_test = y_trn[train_index], y_trn[test_index]
        
        
        classifier.fit(x_train, y_train)
        y_pred = classifier.predict(x_test)
        correct_preds = np.sum(y_pred == y_test)
        e = 1 - (correct_preds / len(y_test))
        
        outputs.append((classifier, e))
    
    # Return the paired (model and error) for all folds
    return outputs # [(model1, error1), (model2, error2), ..., (modelK, errorK)]
    



Question 10

In [107]:
# Usage for the cross_validation function:
from sklearn.tree import DecisionTreeClassifier

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']

X_trn = X_trn.reshape((6000, 3 * 29 * 29))

print(" MAX_DEPTH | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR")
for max_depth in [1, 3, 6, 9, 12, 14]:
    classifier = Classifier(DecisionTreeClassifier(max_depth=max_depth))
    N = 5
    outputs = cross_validation(classifier, 
                           X_trn, 
                           y_trn, 
                           n_folds=N)
    print(f" {max_depth} | ", end="")
    for out_ in outputs:
        print(f"{out_[1]} | ", end="")
    avg_error = np.mean([out_[1] for out_ in outputs])
    print(f"{avg_error}")
    


 MAX_DEPTH | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR
 1 | 0.6541666666666667 | 0.6616666666666666 | 0.6358333333333333 | 0.6375 | 0.63 | 0.6438333333333333
 3 | 0.5275000000000001 | 0.5691666666666666 | 0.565 | 0.5475 | 0.54 | 0.5498333333333333
 6 | 0.4866666666666667 | 0.4933333333333333 | 0.4966666666666667 | 0.48250000000000004 | 0.4783333333333334 | 0.4875
 9 | 0.4675 | 0.4966666666666667 | 0.4783333333333334 | 0.4691666666666666 | 0.45833333333333337 | 0.47400000000000003
 12 | 0.46499999999999997 | 0.4966666666666667 | 0.475 | 0.4675 | 0.4675 | 0.47433333333333333
 14 | 0.47916666666666663 | 0.5058333333333334 | 0.475 | 0.4633333333333334 | 0.4766666666666667 | 0.48


Question 11

In [109]:
from sklearn.linear_model import LogisticRegression

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']

X_trn = X_trn.reshape((6000, 3 * 29 * 29))
# y_trn = y_trn.reshape((len(y_trn), -1)) # OH: CHECK THIS, changes the answer -> needed to work


print(" LAMBDA | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR")
for lambda_val in [0.1, 1, 10, 100, 1000]:
    classifier = Classifier(LogisticRegression(penalty='l2', C=1/lambda_val, max_iter=10000, tol=.001))
    N = 5
    outputs = cross_validation(classifier, 
                           X_trn, 
                           y_trn, 
                           n_folds=N)
    print(f" {lambda_val} | ", end="")
    for out_ in outputs:
        print(f"{out_[1]} | ", end="")
    avg_error = np.mean([out_[1] for out_ in outputs]) 
    print(f"{avg_error}")

# OH: SHOULDNT ERRORS INCREASE WITH LAMBDA? CHECK THIS

 LAMBDA | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR
 0.1 | 0.4408333333333333 | 0.4491666666666667 | 0.46166666666666667 | 0.46166666666666667 | 0.45166666666666666 | 0.45299999999999996
 1 | 0.4158333333333334 | 0.4391666666666667 | 0.43500000000000005 | 0.43333333333333335 | 0.4066666666666666 | 0.426
 10 | 0.38583333333333336 | 0.405 | 0.39416666666666667 | 0.4008333333333334 | 0.3533333333333334 | 0.38783333333333336
 100 | 0.3433333333333334 | 0.38 | 0.345 | 0.3616666666666667 | 0.3308333333333333 | 0.3521666666666667
 1000 | 0.3458333333333333 | 0.37 | 0.3533333333333334 | 0.35583333333333333 | 0.3325 | 0.35150000000000003


Question 12

In [110]:
class KNNClassifier():
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors
        return

    def fit(self, X_trn, y_trn):
        # Just store X_trn, y_trn
        self.model = (X_trn, y_trn)

    def predict(self, X_val, y_val=None):
        assert hasattr(self, "model"), "No fitted model!"
        # do stuff here (use self.model for prediction)
        
        def KNN_predict_(X_trn, y_trn, x, K):
            # Dictionary to store n: distance pairs
            distances = {}
            for n in range(len(X_trn)):
                # Calculate distance between x and x[n]
                distance = np.sqrt(np.sum((x - X_trn[n])**2))
                distances[n] = distance
            
            # Sort distances in ascending order
            sorted_distances = sorted(distances.items(), key=lambda x: x[1])

            # Get the K nearest neighbors
            y_neighbor = []
            for n in range(K):
                n_nearest = sorted_distances[n][0]
                y_neighbor.append(y_trn[n_nearest])
            
            # get majority class
            c_pred = np.argmax(np.bincount(y_neighbor))
            return c_pred
        
        X_trn, y_trn = self.model
        y_pred = []
        for x in X_val:
            y_pred.append(KNN_predict_(X_trn, y_trn, x, self.n_neighbors))

        return y_pred




In [111]:
k_neighbors = [1, 3, 5, 7, 9, 11]

print(" K | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR")
for k in k_neighbors:
    knn = KNNClassifier(k)
    knn.fit(X_trn, y_trn)
    models = cross_validation(knn, X_trn, y_trn, n_folds=5)
    print(f" {k} | ", end="")
    for model in models:
        print(f"{model[1]} | ", end="")
    avg_error = np.mean([model[1] for model in models])
    print(f"{avg_error}")

    

 K | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR
 1 | 0.47 | 0.45833333333333337 | 0.4575 | 0.4408333333333333 | 0.4575 | 0.4568333333333333
 3 | 0.48250000000000004 | 0.45833333333333337 | 0.4658333333333333 | 0.4541666666666667 | 0.4866666666666667 | 0.46950000000000003
 5 | 0.45499999999999996 | 0.44166666666666665 | 0.45166666666666666 | 0.4325 | 0.46499999999999997 | 0.4491666666666666
 7 | 0.43833333333333335 | 0.4291666666666667 | 0.44666666666666666 | 0.42666666666666664 | 0.44833333333333336 | 0.4378333333333333
 9 | 0.43333333333333335 | 0.4258333333333333 | 0.43333333333333335 | 0.43333333333333335 | 0.44499999999999995 | 0.4341666666666667
 11 | 0.4308333333333333 | 0.43500000000000005 | 0.4408333333333333 | 0.4275 | 0.4308333333333333 | 0.433


Question 13


model | hyper-parameters | cross-validation avg. error | public leaderboard accuracy

Classification Tree | max_depth = 9 | .474 | x

Logistic Regression | lambda = 10 | .388 | x

KNN Classification | K = 7 | 0.438 | x

Reasoning:
 - 

In [ ]:
import numpy as np
import csv

def write_csv(y_pred, filename):
    """Write a 1d numpy array to a Kaggle-compatible .csv file"""
    with open(filename, 'w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Id', 'Category'])
        for idx, y in enumerate(y_pred):
            csv_writer.writerow([idx, y])

In [113]:
# Train best tree model on test data and save predictions

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
X_tst = data['X_tst']

X_trn = X_trn.reshape((len(X_trn), 3 * 29 * 29))
X_tst = X_tst.reshape((len(X_tst), 3 * 29 * 29))

clf = DecisionTreeClassifier(max_depth = 9)
clf.fit(X_trn, y_trn)
y_pred = clf.predict(X_tst)
write_csv(y_pred, 'tree_predictions.csv')

In [114]:
# Train best logistic regression model on test data and save predictions

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
X_tst = data['X_tst']

X_trn = X_trn.reshape((len(X_trn), 3 * 29 * 29))
X_tst = X_tst.reshape((len(X_tst), 3 * 29 * 29))

lambda_val = 10
clf = LogisticRegression(penalty='l2', C=1/lambda_val, max_iter=10000, tol=.001)
clf.fit(X_trn, y_trn)
y_pred = clf.predict(X_tst)
write_csv(y_pred, 'logistic_predictions.csv')

In [115]:
# Train best k nearest neighbor on test data and save predictions

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
X_tst = data['X_tst']

X_trn = X_trn.reshape((len(X_trn), 3 * 29 * 29))
X_tst = X_tst.reshape((len(X_tst), 3 * 29 * 29))

k = 7
clf = KNNClassifier(k)
clf.fit(X_trn, y_trn)
y_pred = clf.predict(X_tst)
write_csv(y_pred, 'knn_predictions.csv')